<a href="https://colab.research.google.com/github/smbrmoyo/Porsche_work/blob/main/Porsche_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=e47095bf8a603592c58ed1e361fe7abcbc0f3a011400abfdaea9108c30df5925
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
import pandas as pd
def preprocess_datacombined(orders_filtered, orders_options):
    data_combined = pd.merge(orders_filtered, orders_options, left_on="externalvehiclenumber", right_on="externalvehiclenumber", how="left")
    data_combined["option_key"] = data_combined.groupby("externalvehiclenumber").transform(lambda x: ",".join(x))["option_key"]
    data_combined = data_combined.drop_duplicates(subset=["externalvehiclenumber"])
    data_combined = data_combined.reset_index(drop=True)

    return data_combined

def filtering(dataframe, gg_id, btyp, mj_4):
    dataframe = dataframe[(dataframe["gg_id"]==gg_id) & (dataframe["btyp"]==btyp) & (dataframe["mj_4"]== mj_4)]
    return dataframe
 

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import pandas as pd

def get_frequentitemsets(data):
    
    records = []

    for i in range(0, len(data)):
        list_temp = list(data["option_key"][i].split(","))
        records.append(list_temp)
        
    te = TransactionEncoder()
    te_ary = te.fit(records).transform(records)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True, max_len=1)
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets["itemsets"]=frequent_itemsets["itemsets"].apply(lambda x: list(x)[0])

    return frequent_itemsets

In [ ]:

# Gradient Boosting Regression

from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

params = {'n_estimators': 500,
          'max_depth': 3,
          'min_samples_split': 5,
          'learning_rate': 0.1,
          'loss': 'ls'}

def pred_gbr(train_x, train_y):
    reg = GradientBoostingRegressor(**params)
    reg.fit(train_x, train_y)
    return reg

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor
import numpy as np

def pred_rf(train_x, train_y):
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    rf.fit(train_x, train_y)

    return rf

In [ ]:
# Linear Regression"

from sklearn.linear_model import LinearRegression
import numpy as np

def pred_lr(train_x, train_y):
    lm = LinearRegression()
    lm.fit(train_x, train_y)

    return lm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
from sklearn.metrics import precision_score
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statistics
from sklearn.metrics import mean_squared_error 

#Widgets\n",
import ipywidgets as widgets
from IPython.display import display

#Methoden\n",
#import FrequentItemsets
#import Recommender
#import Preprocessin

#Modelle\n",
#import RandomForest
#import GradientBoosting
#import LinearRegression

btyp = '9YAAA1'
gg_id = 11
mj_4 = 2020

orders = pd.read_csv("drive/MyDrive/orders.csv" , delimiter=";")
orders_options = pd.read_csv("drive/MyDrive/orders_options.csv" , delimiter=";")
option_prices = pd.read_csv("drive/MyDrive/option_package_prices.csv" , delimiter=";")
btyp_prices = pd.read_csv("drive/MyDrive/model_base_price.csv" , delimiter=";")
model_abzug = pd.read_csv("drive/MyDrive/s_abzug_po.csv" , delimiter=";")

In [ ]:
len(model_abzug)

22501

In [ ]:
model_abzug['m_familie'].value_counts()

RAD    780
EIH    592
REL    564
LRA    500
GKH    468
      ... 
087      1
FR       1
21       1
FBK      1
TJ       1
Name: m_familie, Length: 571, dtype: int64

In [ ]:
model_abzug = model_abzug[ (model_abzug["btyp"]==btyp)]
len(model_abzug)

294

In [ ]:
orders_filtered = filtering(orders, gg_id, btyp, mj_4)

In [ ]:
data_combined = preprocess_datacombined(orders_filtered, orders_options)
frequent_itemsets = get_frequentitemsets(data_combined)

In [ ]:
option_prices_filtered = filtering(option_prices, gg_id, btyp, mj_4)
btyp_prices_filtered = filtering(btyp_prices, gg_id, btyp, mj_4)

In [ ]:
support_option_price = pd.merge(option_prices_filtered, frequent_itemsets, left_on="option_key", right_on="itemsets")
support_price = pd.merge(btyp_prices_filtered, support_option_price, on="btyp")

In [ ]:
labels = support_price['support'].values
features = support_price[['preis','gr_prs']].values
features

In [ ]:
corr_matrix = support_price.corr()
corr_matrix["support"].sort_values(ascending=False)

support    1.000000
preis     -0.117786
gg_id_x         NaN
mj_4_x          NaN
gr_prs          NaN
gg_id_y         NaN
mj_4_y          NaN
length          NaN
Name: support, dtype: float64

In [ ]:
from sklearn.preprocessing import MinMaxScaler
cols_to_scale = ['preis','gr_prs']
scaler = MinMaxScaler()
support_price[cols_to_scale] = scaler.fit_transform(support_price[cols_to_scale])
support_price

,row ID_x,gg_id_x,c_nr_x,btyp,mj_4_x,mj_1_x,preis_typ_x,gr_prs,waehr_kz_x,gltg_ab_x,gltg_bis_x,l_aend_x,loaddate_x,row ID_y,gg_id_y,c_nr_y,mj_4_y,mj_1_y,option_key,ausstatt_kz,ausstatt_alt,v_option_key,v_ausstatt_kz,v_ausstatt_alt,preis_typ_y,preis,waehr_kz_y,gltg_ab_y,gltg_bis_y,l_aend_y,loaddate_y,support,itemsets,length
0,Row195,11,C00,9YAAA1,2020,L,RETAIL,0.0,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-06-19T16:05:00.0,2020-08-25T00:00:00.0,Row60404,11,C00,2020,L,C_C_YAV,C,YAV,C_C_YAV,C,YAV,RETAIL,0.033113,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-08-25T14:00:12.133,2020-08-25T00:00:00.0,0.002395,C_C_YAV,1
1,Row195,11,C00,9YAAA1,2020,L,RETAIL,0.0,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-06-19T16:05:00.0,2020-08-25T00:00:00.0,Row60405,11,C00,2020,L,C_C_YAW,C,YAW,C_C_YAW,C,YAW,RETAIL,0.064570,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-08-25T14:00:12.133,2020-08-25T00:00:00.0,0.007186,C_C_YAW,1
2,Row195,11,C00,9YAAA1,2020,L,RETAIL,0.0,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-06-19T16:05:00.0,2020-08-25T00:00:00.0,Row60407,11,C00,2020,L,C_C_YBZ,C,YBZ,C_C_YBZ,C,YBZ,RETAIL,0.028146,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-08-25T14:00:12.133,2020-08-25T00:00:00.0,0.001198,C_C_YBZ,1
3,Row195,11,C00,9YAAA1,2020,L,RETAIL,0.0,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-06-19T16:05:00.0,2020-08-25T00:00:00.0,Row60414,11,C00,2020,L,C_C_YFL,C,YFL,C_C_YFL,C,YFL,RETAIL,0.051325,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-08-25T14:00:12.133,2020-08-25T00:00:00.0,0.010778,C_C_YFL,1
4,Row195,11,C00,9YAAA1,2020,L,RETAIL,0.0,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-06-19T16:05:00.0,2020-08-25T00:00:00.0,Row60416,11,C00,2020,L,C_C_YFX,C,YFX,C_C_YFX,C,YFX,RETAIL,0.062086,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-08-25T14:00:12.133,2020-08-25T00:00:00.0,0.003593,C_C_YFX,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Row195,11,C00,9YAAA1,2020,L,RETAIL,0.0,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-06-19T16:05:00.0,2020-08-25T00:00:00.0,Row60655,11,C00,2020,L,P_P_PO1,P,PO1,P_P_PO1,P,PO1,RETAIL,0.248344,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-08-25T14:00:12.133,2020-08-25T00:00:00.0,0.010778,P_P_PO1,1
218,Row195,11,C00,9YAAA1,2020,L,RETAIL,0.0,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-06-19T16:05:00.0,2020-08-25T00:00:00.0,Row60656,11,C00,2020,L,P_P_PW1,P,PW1,P_P_PW1,P,PW1,RETAIL,0.153974,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-08-25T14:00:12.133,2020-08-25T00:00:00.0,0.245509,P_P_PW1,1
219,Row195,11,C00,9YAAA1,2020,L,RETAIL,0.0,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-06-19T16:05:00.0,2020-08-25T00:00:00.0,Row60657,11,C00,2020,L,P_P_PW5,P,PW5,P_P_PW5,P,PW5,RETAIL,0.236755,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-08-25T14:00:12.133,2020-08-25T00:00:00.0,0.051497,P_P_PW5,1
220,Row195,11,C00,9YAAA1,2020,L,RETAIL,0.0,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-06-19T16:05:00.0,2020-08-25T00:00:00.0,Row60658,11,C00,2020,L,P_P_PW7,P,PW7,P_P_PW7,P,PW7,RETAIL,0.390728,EUR,1970-01-01T00:00:00.0,2030-12-31T00:00:00.0,2020-08-25T14:00:12.133,2020-08-25T00:00:00.0,0.220359,P_P_PW7,1


In [ ]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size = 0.25, random_state = 42)

#dardScaler()
#train_x = sc.fit_transform(train_x)\n",
#test_x = sc.transform(test_x)"

In [ ]:
train_x.shape

(166, 2)

In [ ]:
train_y.shape

(166,)

In [ ]:
import tensorflow as tf
from tensorflow import keras


model = keras.Sequential([
    keras.layers.Dense(166 , activation='relu'),
    keras.layers.Dense(322 , activation='relu'),
    keras.layers.Dense(166 , activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

opt = tf.keras.optimizers.SGD(learning_rate=0.0001)
cce = tf.keras.losses.CategoricalCrossentropy()

model.compile(optimizer=opt,
              loss='mse',
              metrics=['accuracy'])

model.fit(train_x, train_y,epochs=5)

Epoch 1/5
6/6 [==============================] - 0s 4ms/step - loss: 0.0502 - accuracy: 0.0000e+00
Epoch 2/5
6/6 [==============================] - 0s 3ms/step - loss: 0.0487 - accuracy: 0.0000e+00
Epoch 3/5
6/6 [==============================] - 0s 3ms/step - loss: 0.0581 - accuracy: 0.0000e+00
Epoch 4/5
6/6 [==============================] - 0s 3ms/step - loss: 0.0609 - accuracy: 0.0000e+00
Epoch 5/5
6/6 [==============================] - 0s 3ms/step - loss: 0.0495 - accuracy: 0.0000e+00


In [ ]:
labels_list = list(labels)

baseline_preds = statistics.mean(labels_list)
   
baseline_errors = abs(baseline_preds - test_y)
print('Average baseline error: ', round(np.mean(baseline_errors), 3))

predictor_rf = pred_rf(train_x, train_y)

predictions_rf = predictor_rf.predict(test_x)

mse = mean_squared_error(predictions_rf, test_y)

print('MSE:', round(mse, 3))

Average baseline error:  0.124
MSE: 0.028


In [ ]:
predictor_gbr = pred_gbr(train_x, train_y)

predictions_gbr = predictor_gbr.predict(test_x)

mse = mean_squared_error(predictions_gbr, test_y)

print('MSE:', round(mse, 3))

MSE: 0.034


In [ ]:
predictor_lr = pred_lr(train_x, train_y)

predictions_lr = predictor_lr.predict(test_x)

mse = mean_squared_error(predictions_lr, test_y)

print('MSE:', round(mse, 3))

MSE: 0.027


In [ ]:
preis_eingabe = widgets.IntText(description="Optionspreis:", continuous_update=True); display(preis_eingabe)

derivat_preis_eingabe = widgets.IntText(description="Grundpreis Derivat", continuous_update=True); display(derivat_preis_eingabe)

preis = int(preis_eingabe.value)
derivat_preis = int(derivat_preis_eingabe.value)
tr = predictor_lr.predict([[preis, derivat_preis]])
button = widgets.Button(description = "Take Rate berechnen")
output = widgets.Output()
display(button,output)\

def recommender(__init__):
    with output:
        print("Die ermittlete Take Rate ist " + str(tr))

button.on_click(recommender)

IntText(value=0, continuous_update=True, description='Optionspreis:')

IntText(value=0, continuous_update=True, description='Grundpreis Derivat')

Button(description='Take Rate berechnen', style=ButtonStyle())

Output()

In [ ]:
New_data = pd.read_csv("drive/MyDrive/New_dataset.csv",delimiter=',')
New_data

,Unnamed: 0,gg_id_x,btyp,mj_4_x,mj_1_x,gr_prs,waehr_kz_x,option_key,ausstatt_kz,ausstatt_alt,preis,support,pe,gueltig_bis,serienbeginn,m_familie,status_der_mnr,mnrk_1
0,93,11,9YAAA1,2019,K,62881.0,EUR,M_M_0M5,M,0M5,100.0,0.943962,0150_L_FE3BA_9YAAA1_856675_F_FC00,2020-08-16,2019-07-08,KBV,F,NaN
1,167,11,9YAAA1,2019,K,62881.0,EUR,M_M_QZ7,M,QZ7,240.0,0.880104,0150_L_FE3BA_9YAAA1_857525_F_FC00,2020-08-16,2019-07-08,LEN,F,NaN
2,111,11,9YAAA1,2019,K,62881.0,EUR,M_M_3FU,M,3FU,1690.0,0.762815,0150_L_FE3BA_9YAAA1_856879_F_FC00,2020-08-16,2019-07-08,DEI,F,NaN
3,96,11,9YAAA1,2019,K,62881.0,EUR,M_M_0TD,M,0TD,150.0,0.753258,0150_L_FE3BA_9YAAA1_856701_F_FC00,2020-08-16,2019-07-08,ZFM,F,NaN
4,173,11,9YAAA1,2019,K,62881.0,EUR,M_M_VW1,M,VW1,420.0,0.729366,0150_L_FE3BA_9YAAA1_876685_F_FC00,2020-08-16,2019-07-08,SSH,F,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,190,11,9YAAA1,2020,L,64446.0,EUR,M_M_S6C,M,S6C,850.0,0.001198,0150_L_FE3BA_9YAAA1_890LH9_F_FC00,2020-08-16,2019-07-08,S6C,F,NaN
369,93,11,9YAAA1,2020,L,64446.0,EUR,M_E_7TL,M,7TL,1370.0,0.001198,0150_L_FE3BA_9YAAA1_891264_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN
370,79,11,9YAAA1,2020,L,64446.0,EUR,M_E_48Z,M,48Z,5055.0,0.001198,0150_L_FE3BA_9YAAA1_89035J_F_FC00,2020-08-16,2019-07-08,RAD,F,NaN
371,95,11,9YAAA1,2020,L,64446.0,EUR,M_E_7V2,M,7V2,290.0,0.001198,0150_L_FE3BA_9YAAA1_8953XR_F_FC00,2020-08-16,2020-04-06,BEL,F,NaN


In [ ]:
New_data.dtypes

Unnamed: 0          int64
gg_id_x             int64
btyp               object
mj_4_x              int64
mj_1_x             object
gr_prs            float64
waehr_kz_x         object
option_key         object
ausstatt_kz        object
ausstatt_alt       object
preis             float64
support           float64
pe                 object
gueltig_bis        object
serienbeginn       object
m_familie          object
status_der_mnr     object
mnrk_1            float64
dtype: object

In [ ]:
New_data['m_familie'].value_counts()

RAD    32
NNN    14
EIH    14
LRA     9
SZU     9
       ..
HKA     1
S6C     1
GRA     1
S2S     1
ABA     1
Name: m_familie, Length: 95, dtype: int64

In [ ]:
New_data[New_data['m_familie']=='EIH']

,Unnamed: 0,gg_id_x,btyp,mj_4_x,mj_1_x,gr_prs,waehr_kz_x,option_key,ausstatt_kz,ausstatt_alt,preis,support,pe,gueltig_bis,serienbeginn,m_familie,status_der_mnr,mnrk_1
83,136,11,9YAAA1,2019,K,62881.0,EUR,M_M_5TE,M,5TE,650.0,0.061251,0150_L_FE3BA_9YAAA1_857043_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN
105,135,11,9YAAA1,2019,K,62881.0,EUR,M_M_5MP,M,5MP,650.0,0.033015,0150_L_FE3BA_9YAAA1_857029_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN
113,134,11,9YAAA1,2019,K,62881.0,EUR,M_M_5MH,M,5MH,1370.0,0.026933,0150_L_FE3BA_9YAAA1_857027_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN
139,77,11,9YAAA1,2019,K,62881.0,EUR,M_E_7TM,M,7TM,1500.0,0.011729,0150_L_FE3BA_9YAAA1_891266_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN
161,65,11,9YAAA1,2019,K,62881.0,EUR,M_E_5TT,M,5TT,1690.0,0.003041,0150_L_FE3BA_9YAAA1_891260_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN
167,137,11,9YAAA1,2019,K,62881.0,EUR,M_M_5TK,M,5TK,650.0,0.002172,0150_L_FE3BA_9YAAA1_857045_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN
170,76,11,9YAAA1,2019,K,62881.0,EUR,M_E_7TL,M,7TL,1370.0,0.001738,0150_L_FE3BA_9YAAA1_891264_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN
267,157,11,9YAAA1,2020,L,64446.0,EUR,M_M_5TE,M,5TE,650.0,0.044311,0150_L_FE3BA_9YAAA1_857043_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN
289,154,11,9YAAA1,2020,L,64446.0,EUR,M_M_5MH,M,5MH,1370.0,0.026347,0150_L_FE3BA_9YAAA1_857027_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN
303,155,11,9YAAA1,2020,L,64446.0,EUR,M_M_5MP,M,5MP,650.0,0.020359,0150_L_FE3BA_9YAAA1_857029_F_FC00,2020-08-16,2019-07-08,EIH,F,NaN


In [ ]:
New_data[(New_data["m_familie"]=='RAD')].iloc[1]

Unnamed: 0                                      117
gg_id_x                                          11
btyp                                         9YAAA1
mj_4_x                                         2019
mj_1_x                                            K
gr_prs                                        62881
waehr_kz_x                                      EUR
option_key                                  M_M_42K
ausstatt_kz                                       M
ausstatt_alt                                    42K
preis                                          3405
support                                    0.107298
pe                0150_L_FE3BA_9YAAA1_890C94_F_FC00
gueltig_bis                              2020-08-16
serienbeginn                             2019-07-08
m_familie                                       RAD
status_der_mnr                                    F
mnrk_1                                          NaN
Name: 57, dtype: object

In [ ]:
New_data["m_familie"].value_counts()

RAD    32
NNN    14
EIH    14
LRA     9
SZU     9
       ..
HKA     1
S6C     1
GRA     1
S2S     1
ABA     1
Name: m_familie, Length: 95, dtype: int64

In [ ]:
found = []
def find_change_in_fam(df, col, value):

  for i in range(len(df[(df[col]==value)])):
    for j in range(len(df[(df[col]==value)])):
      if df[(New_data[col]==value)].iloc[i]['ausstatt_alt']==df[(df[col]==value)].iloc[j]['ausstatt_alt'] and df[(df[col]==value)].iloc[i]['preis'] == df[(df[col]==value)].iloc[j]['preis'] and df[(df[col]==value)].iloc[i]['mj_4_x'] != df[(df[col]==value)].iloc[j]['mj_4_x'] and i!=j:
      
        if df[(df[col]==value)].iloc[i]['ausstatt_alt'] not in found:
          found.extend([df[(df[col]==value)].iloc[i]['ausstatt_alt'], df[(df[col]==value)].iloc[i]['preis']])
  return found

In [ ]:
print(find_change_in_fam(New_data,'m_familie','TVE'))

['QV3', 400.0]


In [ ]:
a = 0
for ele in New_data["m_familie"].unique().tolist():
  print(ele)

KBV
LEN
DEI
ZFM
SSH
DAE
IRS
CHR
SPU
SIH
LSE
KSU
TVE
LCP
VOS
LRA
2T
SZU
RAU
AM
RAD
VTV
HSW
DAR
SAB
AL
AHV
GRT
EST
HUD
TKV
ZUH
ZIN
KMS
3M
HIA
AER
SBR
SIE
A1
FRI
WSS
QM
S9Y
0Q
KOV
CM
BBO
SSR
C7
EIH
LBH
4A
CDW
GEF
SZ8
FEU
STF
RAA
RM
2Y
RL
HT
CB
CWV
FFW
FVS
AUD
TGE
MAS
HR
BAV
HIM
RER
NNN
Y8
S3S
4T
EIL
CU
CR
TDA
FIT
ZUR
GRA
SAG
S2S
HKA
ABA
F0
3H
5Q
H2
S6C
BEL


In [ ]:
for ele in New_data["m_familie"].unique().tolist():
  find_change_in_fam(New_data,'m_familie',ele)
len(found)

268

In [ ]:
New_data.drop(['status_der_mnr','mnrk_1'], axis=columns, inplace=True)